In [2]:
import requests
import json
from bs4 import BeautifulSoup
import urllib.parse
import pandas as pd

In [31]:
# https://partner.steamgames.com/doc/store/getreviews
# url = 'store.steampowered.com/appreviews/1850570?json=1?'
# https://api.steampowered.com/<interface>/<method>/v<version>/

In [4]:
# https://partner.steamgames.com/doc/store/getreviews
r = requests.get('https://store.steampowered.com/appreviews/252950?json=1&language=all')

In [5]:
# seeing what the JSON looks like
r.json()

{'success': 1,
 'query_summary': {'num_reviews': 20,
  'review_score': 8,
  'review_score_desc': 'Very Positive',
  'total_positive': 365516,
  'total_negative': 40166,
  'total_reviews': 405682},
 'reviews': [{'recommendationid': '111848022',
   'author': {'steamid': '76561198253049543',
    'num_games_owned': 241,
    'num_reviews': 2,
    'playtime_forever': 114932,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 114932,
    'last_played': 1646809697},
   'language': 'english',
   'review': 'This game is slowly but surely getting worse.  I don\'t even care about the overpriced cosmetics because I just don\'t buy them nor use them.  The community has been even more toxic than before and the fact you have to concede in a casual match is ridiculous not to mention you get punished when you leave a "CASUAL" match (your replaced by a damn bot for god sake not like they\'re left high and dry)  The more I play the game the more I want to just uninstall and never look back.  Toda

In [53]:
r = requests.get('https://store.steampowered.com/appreviews/1850570?json=1&language=english&cursor=*&num_per_page=100')
r.json()

{'success': 1,
 'query_summary': {'num_reviews': 100,
  'review_score': 8,
  'review_score_desc': 'Very Positive',
  'total_positive': 426,
  'total_negative': 36,
  'total_reviews': 462},
 'reviews': [{'recommendationid': '113072512',
   'author': {'steamid': '76561198986421535',
    'num_games_owned': 666,
    'num_reviews': 6,
    'playtime_forever': 491,
    'playtime_last_two_weeks': 491,
    'playtime_at_review': 18,
    'last_played': 1648725796},
   'language': 'english',
   'review': '\u2063[h1]List of everthing new in Death Stranding Directors Cut\u2063[/h1]\n\nInfo:\nThe new content is unlocked by progressing through the game.\n\nNew graphics technology:\nImproved water effects\nAMD FidelityFX Super Resolution\nNVIDIA DLSS 2.3.7\nNVIDIA Image Scaling\nIntel XeSS\n\nNew controller functions:\nDualSense Haptic Feedback\n\nNew story missions:\n2 missions available in chapter 2\n1 mission available after finishing the game\n\nNew standard orders:\n40 new orders\n\nNew location:\

In [21]:
# Have to parse the quote, otherwise the URL breaks (browser leads to a dead page)
safe_string = urllib.parse.quote_plus('AoIIP0pRwn+UsK8D')
# print(safe_string)
r = requests.get(f'https://store.steampowered.com/appreviews/1850570?json=1&language=english&cursor={safe_string}')
r.json()


{'success': 1,
 'query_summary': {'num_reviews': 20},
 'reviews': [{'recommendationid': '113095872',
   'author': {'steamid': '76561197986511953',
    'num_games_owned': 733,
    'num_reviews': 313,
    'playtime_forever': 1260,
    'playtime_last_two_weeks': 1260,
    'playtime_at_review': 305,
    'last_played': 1649152031},
   'language': 'english',
   'review': 'Death Stranding is the newest game from Hideo Kojima. Fresh from the shackles of Konami he decided to create a game without any creative or financial restrictions. That project ended up being Death Stranding. In many ways it\'s an extremely creative and bizarre game that has a very unique appeal and core gameplay loop you cannot find anywhere else.\n\nDeath Stranding takes place in a post apocalyptic USA set after an event aptly named "The Death Stranding". Which caused apocalyptic explosions to appear all over the world, harmful rain known as Timefall which makes substances and people age upon contact with it, and these gh

In [58]:
def get_reviews(appid, cursor, filter_='recent', review_type='all'):
    """Gets steam review in JSON
        Returns JSON & cursor for next set of reviews
    """

    if cursor != '*':
        cursor = urllib.parse.quote_plus(cursor)
        
    r = requests.get(f'https://store.steampowered.com/appreviews/{str(appid)}?json=1&language=english&cursor={cursor}&num_per_page=100&filter={filter_}&review_type={review_type}')
    return r.json(), r.json()['cursor']

In [27]:
appid = 1850570
cursor = '*'

data, c = get_reviews(appid, cursor)

({'success': 1,
  'query_summary': {'num_reviews': 20,
   'review_score': 8,
   'review_score_desc': 'Very Positive',
   'total_positive': 424,
   'total_negative': 36,
   'total_reviews': 460},
  'reviews': [{'recommendationid': '113072512',
    'author': {'steamid': '76561198986421535',
     'num_games_owned': 666,
     'num_reviews': 6,
     'playtime_forever': 491,
     'playtime_last_two_weeks': 491,
     'playtime_at_review': 18,
     'last_played': 1648725796},
    'language': 'english',
    'review': '\u2063[h1]List of everthing new in Death Stranding Directors Cut\u2063[/h1]\n\nInfo:\nThe new content is unlocked by progressing through the game.\n\nNew graphics technology:\nImproved water effects\nAMD FidelityFX Super Resolution\nNVIDIA DLSS 2.3.7\nNVIDIA Image Scaling\nIntel XeSS\n\nNew controller functions:\nDualSense Haptic Feedback\n\nNew story missions:\n2 missions available in chapter 2\n1 mission available after finishing the game\n\nNew standard orders:\n40 new orders\n

In [78]:
# pd.DataFrame({'Death Stranding': data[0]['query_summary']})
data[0]['query_summary']['Game Title'] = 'Death Stranding'
sample_df = data[0]['query_summary']

pd.DataFrame.from_dict(sample_df, orient='index') 

,0
num_reviews,20
review_score,8
review_score_desc,Very Positive
total_positive,424
total_negative,36
total_reviews,460
Game Title,Death Stranding


In [28]:
appid = 1850570
cursor = '*'
data = []
for i in range(10):
    review_json, new_cursor = get_reviews(appid, cursor)
    data.append(review_json)
    cursor = new_cursor

In [91]:
# data

In [90]:
# pd.DataFrame.from_dict(data)

In [89]:
# pd.DataFrame.from_dict(data[0]['reviews'])

In [42]:
df = pd.DataFrame()
for i in range(len(data)):
    review_data = pd.DataFrame.from_dict(data[i]['reviews'])
    df = pd.concat([df, review_data])

In [46]:
def json_to_df(raw_json, game_title):
    """
    Takes raw json data and converts it to dataframe & adds game title column
    returns dataframe ready to export to csv
    """
    df = pd.DataFrame()
    
    for i in range(len(raw_json)):
        review_data = pd.DataFrame.from_dict(raw_json[i]['reviews'])
        df = pd.concat([df, review_data])
    df['game_title'] = game_title
    
    return df

In [47]:
def query_summary(raw_json, game_title):
    """
    Saves Query Summary data as csv
    """
    query_summary = raw_json[0]['query_summary']
    query_summary['Game Title'] = game_title
    out = pd.DataFrame.from_dict(query_summary, orient='index')
    out.to_csv(f"data/{game_title}_query_summary.csv")
    print(f'Saved Query Summary to data/{game_title}_query_summary.csv')

In [49]:
deathstranding = json_to_df(data, "DEATH STRANDING DIRECTOR'S CUT")

In [52]:
deathstranding.to_csv('data/death_stranding.csv')

In [87]:
query_summary(data, 'death_stranding')

Saved Query Summary to data/death_stranding_query_summary.csv


In [56]:
# Get IDs for all the action games
action_ids = [915810, 1911390, 1097350, 1475810, 1051690, 1602010, 920210, 1692070, 1599340, 1498570, 1245620, 1485080, 1461600, 1446780]
action_ids_dict = {
    1850570:'DEATH_STRANDING_DIRECTORS_CUT',
    915810:'Midnight_Ghost_Hunt',
    1911390:'Knightfall_A_Daring_Journey',
    1097350:'Weird_West',
    1475810:'Ghostwire_Tokyo',
    1051690:'Nightmare_Reaper',
    1602010:'Persona_4_Arena_Ultimax',
    920210: 'LEGO_Star_Wars_The_Skywalker_Saga',
    1389990: 'Oblivity__Find_your_perfect_Sensitivity',
    1599340:'Lost_Ark',
    1498570: 'THE_KING_OF_FIGHTERS_XV',
    1245620: 'ELDEN_RING',
    1485080:'Just_Act_Natural',
    1461600:'Project_Sparrow',
    1446780:'MONSTER_HUNTER_RISE',
    1692070: 'CROWZ',
    1593500:'God_of_War',
    534380:'Dying_Light_2_Stay_Human',
    1740720: 'Have_a_Nice_Death',
    553420: 'TUNIC',
    1259420: 'Days_Gone',
}

## Pipeline:
### get_reviews(appid, cursor)
### json_to_df(raw_json, game_title)
####    save df to csv
### query_summary(raw_json, game_title)

In [48]:
# def reviews_pipeline(appid, game_title, n_searches=4, filepath=None):
#     cursor = '*'
#     data = []
#     if filepath == None:
#         filepath = f'data/{game_title}'
    
#     for i in range(n_searches//2):
#         review_json, new_cursor = get_reviews(appid, cursor)
#         data.append(review_json)
#         cursor = new_cursor
        
#     review_data = json_to_df(data, game_title)
#     review_data.to_csv(f'{filepath}.csv')
#     query_summary(data, game_title)
    

In [67]:
def reviews_pipeline(appid, game_title, n_searches=4, filepath=None):
    """
    Pipeline for gathering reviews. Gathers equal numbers of positive and negative reviews.
    Saves review data and query summary as .csv in the filepath
    """
    cursor = '*'
    data = []
    if filepath == None:
        filepath = f'data/{game_title}'
    
    for i in range(n_searches//2):
        review_json, new_cursor = get_reviews(appid, cursor, review_type='positive')
        data.append(review_json)
        cursor = new_cursor
        
    for i in range(n_searches//2):
        review_json, new_cursor = get_reviews(appid, cursor, review_type='negative')
        data.append(review_json)
        cursor = new_cursor
        
    review_data = json_to_df(data, game_title)
    review_data.to_csv(f'{filepath}.csv')
    query_summary(data, game_title)
    

In [153]:
reviews_pipeline(915810, 'Midnight_Ghost_Hunt')

Saved Query Summary to data/Midnight_Ghost_Hunt_query_summary.csv


In [154]:
for k,v in action_ids_dict.items():
    reviews_pipeline(k, v)

Saved Query Summary to data/DEATH_STRANDING_DIRECTORS_CUT_query_summary.csv
Saved Query Summary to data/Midnight_Ghost_Hunt_query_summary.csv
Saved Query Summary to data/Knightfall_A_Daring_Journey_query_summary.csv
Saved Query Summary to data/Weird_West_query_summary.csv
Saved Query Summary to data/Ghostwire_Tokyo_query_summary.csv
Saved Query Summary to data/Nightmare_Reaper_query_summary.csv
Saved Query Summary to data/Persona_4_Arena_Ultimax_query_summary.csv
Saved Query Summary to data/LEGO_Star_Wars_The_Skywalker_Saga_query_summary.csv
Saved Query Summary to data/Oblivity__Find_your_perfect_Sensitivity_query_summary.csv
Saved Query Summary to data/Lost_Ark_query_summary.csv
Saved Query Summary to data/THE_KING_OF_FIGHTERS_XV_query_summary.csv
Saved Query Summary to data/ELDEN_RING_query_summary.csv
Saved Query Summary to data/Just_Act_Natural_query_summary.csv
Saved Query Summary to data/Project_Sparrow_query_summary.csv
Saved Query Summary to data/MONSTER_HUNTER_RISE_query_summa

In [156]:
x = """
1218210/Coromon/
1588530/Dungeon_Alchemist/
1637730/Crystal_Project/
1638390/Indies_Lies/
1617030/tERRORbane/
1814570/Transcendence_Legacy_
1656780/Heros_Hour/
900040/ELEX_II/
1173820/FINAL_FANTASY_VI/
1472660/No_Place_Like_Home/
1621310/Atelier_Sophie_2_The_Alchemist_of_the_Mysterio
1170950/Mortal_Online_2/
1368030/ANNO_Mutationem/
1832640/Mirror_2_Project_X/
987840/Expeditions_Rome/
1826960/Trip_In_Another_World/
1432050/Nobody_Saves_the_World/
1374930/Black_Geyser_Couriers_of_Darkness/
39210/FINAL_FANTASY_XIV_Online/
306130/The_Elder_Scrolls_Online/
1286830/STAR_WARS_The_Old_Republic/
1085660/Destiny_2/
230410/Warframe/
582660/Black_Desert/
"""

In [181]:
def url_to_dict(urls):
    """
    Returns a dict from a string of urls
    """
    x = urls.split('\n')
    x = [i.lstrip('https://store.steampowered.com/app/') for i in x]
    y = [i.rstrip('/').split('/') for i in x]
    z = y[1:-1]
    out = dict(z)
    return out

url_to_dict(x)

{'1218210': 'Coromon',
 '1588530': 'Dungeon_Alchemist',
 '1637730': 'Crystal_Project',
 '1638390': 'Indies_Lies',
 '1617030': 'tERRORbane',
 '1814570': 'Transcendence_Legacy_',
 '1656780': 'Heros_Hour',
 '900040': 'ELEX_II',
 '1173820': 'FINAL_FANTASY_VI',
 '1472660': 'No_Place_Like_Home',
 '1621310': 'Atelier_Sophie_2_The_Alchemist_of_the_Mysterio',
 '1170950': 'Mortal_Online_2',
 '1368030': 'ANNO_Mutationem',
 '1832640': 'Mirror_2_Project_X',
 '987840': 'Expeditions_Rome',
 '1826960': 'Trip_In_Another_World',
 '1432050': 'Nobody_Saves_the_World',
 '1374930': 'Black_Geyser_Couriers_of_Darkness',
 '39210': 'FINAL_FANTASY_XIV_Online',
 '306130': 'The_Elder_Scrolls_Online',
 '1286830': 'STAR_WARS_The_Old_Republic',
 '1085660': 'Destiny_2',
 '230410': 'Warframe',
 '582660': 'Black_Desert'}

In [125]:
rpg_ids_dict

[['1218210', 'Coromon'],
 ['1588530', 'Dungeon_Alchemist'],
 ['1637730', 'Crystal_Project'],
 ['1638390', 'Indies_Lies'],
 ['1617030', 'tERRORbane'],
 ['1814570', 'Transcendence_Legacy_'],
 ['1656780', 'Heros_Hour'],
 ['900040', 'ELEX_II'],
 ['1173820', 'FINAL_FANTASY_VI'],
 ['1472660', 'No_Place_Like_Home'],
 ['1621310', 'Atelier_Sophie_2_The_Alchemist_of_the_Mysterio'],
 ['1170950', 'Mortal_Online_2'],
 ['1368030', 'ANNO_Mutationem'],
 ['1832640', 'Mirror_2_Project_X'],
 ['987840', 'Expeditions_Rome'],
 ['1826960', 'Trip_In_Another_World'],
 ['1432050', 'Nobody_Saves_the_World'],
 ['1374930', 'Black_Geyser_Couriers_of_Darkness'],
 ['39210', 'FINAL_FANTASY_XIV_Online'],
 ['306130', 'The_Elder_Scrolls_Online'],
 ['1286830', 'STAR_WARS_The_Old_Republic'],
 ['1085660', 'Destiny_2'],
 ['230410', 'Warframe'],
 ['582660', 'Black_Desert']]

In [155]:
for k,v in rpg_ids_dict.items():
    reviews_pipeline(k, v)

Saved Query Summary to data/Coromon_query_summary.csv
Saved Query Summary to data/Dungeon_Alchemist_query_summary.csv
Saved Query Summary to data/Crystal_Project_query_summary.csv
Saved Query Summary to data/Indies_Lies_query_summary.csv
Saved Query Summary to data/tERRORbane_query_summary.csv
Saved Query Summary to data/Transcendence_Legacy__query_summary.csv
Saved Query Summary to data/Heros_Hour_query_summary.csv
Saved Query Summary to data/ELEX_II_query_summary.csv
Saved Query Summary to data/FINAL_FANTASY_VI_query_summary.csv
Saved Query Summary to data/No_Place_Like_Home_query_summary.csv
Saved Query Summary to data/Atelier_Sophie_2_The_Alchemist_of_the_Mysterio_query_summary.csv
Saved Query Summary to data/Mortal_Online_2_query_summary.csv
Saved Query Summary to data/ANNO_Mutationem_query_summary.csv
Saved Query Summary to data/Mirror_2_Project_X_query_summary.csv
Saved Query Summary to data/Expeditions_Rome_query_summary.csv
Saved Query Summary to data/Trip_In_Another_World_que

In [158]:
strategy_urls = """
https://store.steampowered.com/app/1142710/Total_War_WARHAMMER_III/
https://store.steampowered.com/app/1260520/Patricks_Parabox/
https://store.steampowered.com/app/1138100/Touhou_Ibunseki__Ayaria_Dawn_ReCreation/
https://store.steampowered.com/app/1449850/YuGiOh_Master_Duel/
https://store.steampowered.com/app/960090/Bloons_TD_6/
https://store.steampowered.com/app/1569040/Football_Manager_2022/
https://store.steampowered.com/app/813780/Age_of_Empires_II_Definitive_Edition/
https://store.steampowered.com/app/394360/Hearts_of_Iron_IV/
https://store.steampowered.com/app/1324780/Easy_Red_2/
https://store.steampowered.com/app/1560500/Badlanders/
https://store.steampowered.com/app/1570070/Beholder_3/
https://store.steampowered.com/app/1668690/Alina_of_the_Arena/
https://store.steampowered.com/app/1826960/Trip_In_Another_World/
https://store.steampowered.com/app/1489970/Highrise_City/
https://store.steampowered.com/app/887570/NEBULOUS_Fleet_Command/
https://store.steampowered.com/app/872670/SCP_Pandemic/
https://store.steampowered.com/app/1887930/Idle_Monster_TD_Evolved/
https://store.steampowered.com/app/1428100/Instruments_of_Destruction/
https://store.steampowered.com/app/987840/Expeditions_Rome/
https://store.steampowered.com/app/1220010/Home_Behind_2/
https://store.steampowered.com/app/1410710/Democracy_4/
https://store.steampowered.com/app/982290/Airborne_Kingdom/
https://store.steampowered.com/app/1358710/Battle_Cry_of_Freedom/
https://store.steampowered.com/app/1745400/League_Manager_2022/
https://store.steampowered.com/app/1611600/WARNO/
https://store.steampowered.com/app/1272320/Diplomacy_is_Not_an_Option/
https://store.steampowered.com/app/1531540/Distant_Worlds_2/
https://store.steampowered.com/app/1418630/Dread_Hunger/
"""

In [167]:
strategy_ids = url_to_dict(strategy_urls)

In [168]:
for k,v in strategy_ids.items():
    reviews_pipeline(k, v)

Saved Query Summary to data/Total_War_WARHAMMER_III_query_summary.csv
Saved Query Summary to data/Patricks_Parabox_query_summary.csv
Saved Query Summary to data/Touhou_Ibunseki__Ayaria_Dawn_ReCreation_query_summary.csv
Saved Query Summary to data/YuGiOh_Master_Duel_query_summary.csv
Saved Query Summary to data/Bloons_TD_6_query_summary.csv
Saved Query Summary to data/Football_Manager_2022_query_summary.csv
Saved Query Summary to data/Age_of_Empires_II_Definitive_Edition_query_summary.csv
Saved Query Summary to data/Hearts_of_Iron_IV_query_summary.csv
Saved Query Summary to data/Easy_Red_2_query_summary.csv
Saved Query Summary to data/Badlanders_query_summary.csv
Saved Query Summary to data/Beholder_3_query_summary.csv
Saved Query Summary to data/Alina_of_the_Arena_query_summary.csv
Saved Query Summary to data/Trip_In_Another_World_query_summary.csv
Saved Query Summary to data/Highrise_City_query_summary.csv
Saved Query Summary to data/NEBULOUS_Fleet_Command_query_summary.csv
Saved Quer

In [179]:
adventure_urls = """
https://store.steampowered.com/app/264710/Subnautica/
https://store.steampowered.com/app/252490/Rust/
https://store.steampowered.com/app/242760/The_Forest/
https://store.steampowered.com/app/892970/Valheim/
https://store.steampowered.com/app/367520/Hollow_Knight/
https://store.steampowered.com/app/413150/Stardew_Valley/
https://store.steampowered.com/app/105600/Terraria/
https://store.steampowered.com/app/775500/SCARLET_NEXUS/
https://store.steampowered.com/app/582660/Black_Desert/
https://store.steampowered.com/app/812140/Assassins_Creed_Odyssey/
https://store.steampowered.com/app/346110/ARK_Survival_Evolved/
https://store.steampowered.com/app/1088850/Marvels_Guardians_of_the_Galaxy/
https://store.steampowered.com/app/920210/LEGO_Star_Wars_The_Skywalker_Saga/
https://store.steampowered.com/app/1504500/OPUS_Echo_of_Starsong/
https://store.steampowered.com/app/1172620/Sea_of_Thieves/
https://store.steampowered.com/app/707030/POSTAL_4_No_Regerts/
https://store.steampowered.com/app/361420/ASTRONEER/
https://store.steampowered.com/app/9900/Star_Trek_Online/
https://store.steampowered.com/app/322330/Dont_Starve_Together/
https://store.steampowered.com/app/648800/Raft/
https://store.steampowered.com/app/684450/Surviving_the_Aftermath/
https://store.steampowered.com/app/1843760/Rogue_Tower/
"""

In [183]:
url_to_dict(adventure_urls)

{'264710': 'Subnautica',
 '252490': 'Rust',
 '242760': 'The_Forest',
 '892970': 'Valheim',
 '367520': 'Hollow_Knight',
 '413150': 'Stardew_Valley',
 '105600': 'Terraria',
 '775500': 'SCARLET_NEXUS',
 '582660': 'Black_Desert',
 '812140': 'Assassins_Creed_Odyssey',
 '346110': 'ARK_Survival_Evolved',
 '1088850': 'Marvels_Guardians_of_the_Galaxy',
 '920210': 'LEGO_Star_Wars_The_Skywalker_Saga',
 '1504500': 'OPUS_Echo_of_Starsong',
 '1172620': 'Sea_of_Thieves',
 '707030': 'POSTAL_4_No_Regerts',
 '361420': 'ASTRONEER',
 '9900': 'Star_Trek_Online',
 '322330': 'Dont_Starve_Together',
 '648800': 'Raft',
 '684450': 'Surviving_the_Aftermath',
 '1843760': 'Rogue_Tower'}

In [184]:
adventure_ids = url_to_dict(adventure_urls)
for k,v in adventure_ids.items():
    reviews_pipeline(k, v)

Saved Query Summary to data/Subnautica_query_summary.csv
Saved Query Summary to data/Rust_query_summary.csv
Saved Query Summary to data/The_Forest_query_summary.csv
Saved Query Summary to data/Valheim_query_summary.csv
Saved Query Summary to data/Hollow_Knight_query_summary.csv
Saved Query Summary to data/Stardew_Valley_query_summary.csv
Saved Query Summary to data/Terraria_query_summary.csv
Saved Query Summary to data/SCARLET_NEXUS_query_summary.csv
Saved Query Summary to data/Black_Desert_query_summary.csv
Saved Query Summary to data/Assassins_Creed_Odyssey_query_summary.csv
Saved Query Summary to data/ARK_Survival_Evolved_query_summary.csv
Saved Query Summary to data/Marvels_Guardians_of_the_Galaxy_query_summary.csv
Saved Query Summary to data/LEGO_Star_Wars_The_Skywalker_Saga_query_summary.csv
Saved Query Summary to data/OPUS_Echo_of_Starsong_query_summary.csv
Saved Query Summary to data/Sea_of_Thieves_query_summary.csv
Saved Query Summary to data/POSTAL_4_No_Regerts_query_summary.

In [185]:
simulation_urls = """
https://store.steampowered.com/app/357310/Oriental_Empires/
https://store.steampowered.com/app/1166290/Death_and_Taxes/
https://store.steampowered.com/app/376210/The_Isle/
https://store.steampowered.com/app/431240/Golf_With_Your_Friends/
https://store.steampowered.com/app/286160/Tabletop_Simulator/
https://store.steampowered.com/app/1258080/Shop_Titans/
https://store.steampowered.com/app/493340/Planet_Coaster/
https://store.steampowered.com/app/815370/Green_Hell/
https://store.steampowered.com/app/1222670/The_Sims_4/
https://store.steampowered.com/app/359320/Elite_Dangerous/
https://store.steampowered.com/app/1241510/The_Test/
https://store.steampowered.com/app/518790/theHunter_Call_of_the_Wild/
https://store.steampowered.com/app/936720/Wrench/
https://store.steampowered.com/app/1076750/Dream_Engines_Nomad_Cities__A_survival_city_builder_with_flying_cities/
https://store.steampowered.com/app/789660/Buggos/
https://store.steampowered.com/app/1424330/Wobbledogs/
https://store.steampowered.com/app/982290/Airborne_Kingdom/
https://store.steampowered.com/app/1472660/No_Place_Like_Home/
https://store.steampowered.com/app/233720/Surgeon_Simulator/
https://store.steampowered.com/app/844590/Hypnospace_Outlaw/
https://store.steampowered.com/app/424030/War_of_Rights/
https://store.steampowered.com/app/1149620/Gas_Station_Simulator/
https://store.steampowered.com/app/666140/My_Time_At_Portia/
"""

In [186]:
simulation_ids = url_to_dict(simulation_urls)
for k,v in simulation_ids.items():
    reviews_pipeline(k, v)

Saved Query Summary to data/Oriental_Empires_query_summary.csv
Saved Query Summary to data/Death_and_Taxes_query_summary.csv
Saved Query Summary to data/The_Isle_query_summary.csv
Saved Query Summary to data/Golf_With_Your_Friends_query_summary.csv
Saved Query Summary to data/Tabletop_Simulator_query_summary.csv
Saved Query Summary to data/Shop_Titans_query_summary.csv
Saved Query Summary to data/Planet_Coaster_query_summary.csv
Saved Query Summary to data/Green_Hell_query_summary.csv
Saved Query Summary to data/The_Sims_4_query_summary.csv
Saved Query Summary to data/Elite_Dangerous_query_summary.csv
Saved Query Summary to data/The_Test_query_summary.csv
Saved Query Summary to data/theHunter_Call_of_the_Wild_query_summary.csv
Saved Query Summary to data/Wrench_query_summary.csv
Saved Query Summary to data/Dream_Engines_Nomad_Cities__A_survival_city_builder_with_flying_cities_query_summary.csv
Saved Query Summary to data/Buggos_query_summary.csv
Saved Query Summary to data/Wobbledogs_q

In [187]:
sports_urls = """
https://store.steampowered.com/app/1247360/Grand_Mountain_Adventure_Wonderlands/
https://store.steampowered.com/app/1255630/WWE_2K22/
https://store.steampowered.com/app/1506830/FIFA_22/
https://store.steampowered.com/app/1134570/F1_2021/
https://store.steampowered.com/app/1551360/Forza_Horizon_5/
https://store.steampowered.com/app/1644960/NBA_2K22/
https://store.steampowered.com/app/380600/Fishing_Planet/
https://store.steampowered.com/app/1519350/Madden_NFL_22/
https://store.steampowered.com/app/1222680/Need_for_Speed_Heat/
https://store.steampowered.com/app/1874170/Shredders/
https://store.steampowered.com/app/1307710/GRID_Legends/
https://store.steampowered.com/app/1190170/OlliOlli_World/
https://store.steampowered.com/app/284160/BeamNGdrive/
https://store.steampowered.com/app/266410/iRacing/
https://store.steampowered.com/app/244210/Assetto_Corsa/
https://store.steampowered.com/app/1465360/SnowRunner/
https://store.steampowered.com/app/962730/Skater_XL__The_Ultimate_Skateboarding_Game/
https://store.steampowered.com/app/655500/MX_Bikes/
https://store.steampowered.com/app/805550/Assetto_Corsa_Competizione/
https://store.steampowered.com/app/635260/CarX_Drift_Racing_Online/
https://store.steampowered.com/app/386180/Crossout/
https://store.steampowered.com/app/571740/Golf_It/
"""

In [188]:
sports_ids = url_to_dict(sports_urls)
for k,v in sports_ids.items():
    reviews_pipeline(k, v)

Saved Query Summary to data/Grand_Mountain_Adventure_Wonderlands_query_summary.csv
Saved Query Summary to data/WWE_2K22_query_summary.csv
Saved Query Summary to data/FIFA_22_query_summary.csv
Saved Query Summary to data/F1_2021_query_summary.csv
Saved Query Summary to data/Forza_Horizon_5_query_summary.csv
Saved Query Summary to data/NBA_2K22_query_summary.csv
Saved Query Summary to data/Fishing_Planet_query_summary.csv
Saved Query Summary to data/Madden_NFL_22_query_summary.csv
Saved Query Summary to data/Need_for_Speed_Heat_query_summary.csv
Saved Query Summary to data/Shredders_query_summary.csv
Saved Query Summary to data/GRID_Legends_query_summary.csv
Saved Query Summary to data/OlliOlli_World_query_summary.csv
Saved Query Summary to data/BeamNGdrive_query_summary.csv
Saved Query Summary to data/iRacing_query_summary.csv
Saved Query Summary to data/Assetto_Corsa_query_summary.csv
Saved Query Summary to data/SnowRunner_query_summary.csv
Saved Query Summary to data/Skater_XL__The_Ul

In [9]:
# Gathering data from the top 200 games via Steamcharts.com
# Steamcharts.com tracks the current, peak and total number of players playing games. It's great for tracking popularity
# over time.
r = requests.get('https://steamcharts.com/top/p.8')
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Steam Charts - Tracking What's Played
  </title>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="An ongoing analysis of Steam's player numbers, seeing what's been played the most." name="description"/>
  <meta content="4503599630058963" property="twitter:account_id"/>
  <script async="" site="steam-charts" src="https://kumo.network-n.com/dist/app.js">
  </script>
  <link href="/assets/style-9826f406.css" rel="stylesheet"/>
  <script src="/assets/script-88db269a.js">
  </script>
  <link href="/assets/favicon.ico" rel="shortcut icon"/>
  <link href="/top/p.7" rel="prev">
   <link href="/top/p.9" rel="next">
   </link>
  </link>
 </head>
 <body>
  <div id="nn_skinl">
  </div>
  <div id="nn_skinr">
  </div>
  <div id="content-wrapper">
   <div id="header">
    <a href="/" id="logo">
     <span id="logo-steam">
      STEAM
     </span>
     <span id="logo-charts">
     

In [26]:
all_links = soup.find_all('a')
# for link in all_links:
#     print(link)
# all_links[4].get('href')
all_links[4].get_text().strip('\n\t')

'Pummel Party'

In [36]:
# for link in all_links:
#     href = link.get('href')
#     text = link.get_text().strip('\n\t')
#     print(href, text,'\n')

steam_charts = dict()

for link in all_links:
    href = link.get('href')
    text = link.get_text().strip('\n\t')
    if href.startswith('/app/'):
        print((int(href[5:]), text))
        temp = {int(href[5:]): text}
        steam_charts.update(temp)

(880940, 'Pummel Party')
(8500, 'EVE Online')
(1371580, 'Myth of Empires')
(310950, 'Street Fighter V')
(933110, 'Age of Empires III: Definitive Edition')
(1066780, 'Transport Fever 2')
(392160, 'X4: Foundations')
(49520, 'Borderlands 2')
(220200, 'Kerbal Space Program')
(359320, 'Elite Dangerous')
(387990, 'Scrap Mechanic')
(1343400, 'RuneScape')
(666140, 'My Time At Portia')
(728880, 'Overcooked! 2')
(552500, 'Warhammer: Vermintide 2')
(1476970, 'Legends of Idleon MMO')
(1825750, 'SoulWorker')
(924970, 'Back 4 Blood')
(962130, 'Grounded')
(72850, 'The Elder Scrolls V: Skyrim')
(700330, 'SCP: Secret Laboratory')
(1787090, 'MyDockFinder')
(872410, 'ROMANCE OF THE THREE KINGDOMS XIV')
(1850570, "DEATH STRANDING DIRECTOR'S CUT")
(233860, 'Kenshi')


In [37]:
steam_charts

{880940: 'Pummel Party',
 8500: 'EVE Online',
 1371580: 'Myth of Empires',
 310950: 'Street Fighter V',
 933110: 'Age of Empires III: Definitive Edition',
 1066780: 'Transport Fever 2',
 392160: 'X4: Foundations',
 49520: 'Borderlands 2',
 220200: 'Kerbal Space Program',
 359320: 'Elite Dangerous',
 387990: 'Scrap Mechanic',
 1343400: 'RuneScape',
 666140: 'My Time At Portia',
 728880: 'Overcooked! 2',
 552500: 'Warhammer: Vermintide 2',
 1476970: 'Legends of Idleon MMO',
 1825750: 'SoulWorker',
 924970: 'Back 4 Blood',
 962130: 'Grounded',
 72850: 'The Elder Scrolls V: Skyrim',
 700330: 'SCP: Secret Laboratory',
 1787090: 'MyDockFinder',
 872410: 'ROMANCE OF THE THREE KINGDOMS XIV',
 1850570: "DEATH STRANDING DIRECTOR'S CUT",
 233860: 'Kenshi'}

In [39]:
a = [1,2,3]
b = [4,5,6]
a.extend(b)
a

[1, 2, 3, 4, 5, 6]

In [40]:
def steamcharts(n):
    """
    Returns a dict {appid: game_title} of all the games on steamcharts.com
    """
    steam_charts = dict()
    all_links = []
    for i in range(1, n+1):
        r = requests.get(f'https://steamcharts.com/top/p.{i}')
        soup = BeautifulSoup(r.content, 'html.parser')
        links = soup.find_all('a')
        all_links.extend(links)

    for link in all_links:
        href = link.get('href')
        text = link.get_text().strip('\n\t')
        if href.startswith('/app/'):
            print((int(href[5:]), text))
            temp = {int(href[5:]): text}
            steam_charts.update(temp)
    return steam_charts

In [41]:
# Gathering top 200 most played games on steam
steamcharts_dict = steamcharts(8)

(730, 'Counter-Strike: Global Offensive')
(570, 'Dota 2')
(1599340, 'Lost Ark')
(578080, 'PUBG: BATTLEGROUNDS')
(1245620, 'ELDEN RING')
(1172470, 'Apex Legends')
(271590, 'Grand Theft Auto V')
(252490, 'Rust')
(440, 'Team Fortress 2')
(1569040, 'Football Manager 2022')
(431960, 'Wallpaper Engine')
(1418630, 'Dread Hunger')
(236390, 'War Thunder')
(1623660, 'MIR4')
(1203220, 'NARAKA: BLADEPOINT')
(1085660, 'Destiny 2')
(359550, "Tom Clancy's Rainbow Six Siege")
(346110, 'ARK: Survival Evolved')
(230410, 'Warframe')
(1506830, 'FIFA 22')
(1449850, 'Yu-Gi-Oh!  Master Duel')
(289070, "Sid Meier's Civilization VI")
(394360, 'Hearts of Iron IV')
(920210, 'LEGO® Star Wars™: The Skywalker Saga')
(251570, '7 Days to Die')
(1794680, 'Vampire Survivors')
(221100, 'DayZ')
(381210, 'Dead by Daylight')
(304930, 'Unturned')
(227300, 'Euro Truck Simulator 2')
(444200, 'World of Tanks Blitz')
(4000, "Garry's Mod")
(105600, 'Terraria')
(413150, 'Stardew Valley')
(252950, 'Rocket League')
(1063730, 'New W

In [69]:
# len(steamcharts_dict) # 200 games, as expected
reviews_pipeline(389730, 'TEKKEN 7')

Saved Query Summary to data/TEKKEN 7_query_summary.csv


In [71]:
for k,v in steamcharts_dict.items():
    reviews_pipeline(k, v)

Saved Query Summary to data/Counter-Strike: Global Offensive_query_summary.csv
Saved Query Summary to data/Dota 2_query_summary.csv
Saved Query Summary to data/Lost Ark_query_summary.csv
Saved Query Summary to data/PUBG: BATTLEGROUNDS_query_summary.csv
Saved Query Summary to data/ELDEN RING_query_summary.csv
Saved Query Summary to data/Apex Legends_query_summary.csv
Saved Query Summary to data/Grand Theft Auto V_query_summary.csv
Saved Query Summary to data/Rust_query_summary.csv
Saved Query Summary to data/Team Fortress 2_query_summary.csv
Saved Query Summary to data/Football Manager 2022_query_summary.csv
Saved Query Summary to data/Wallpaper Engine_query_summary.csv
Saved Query Summary to data/Dread Hunger_query_summary.csv
Saved Query Summary to data/War Thunder_query_summary.csv
Saved Query Summary to data/MIR4_query_summary.csv
Saved Query Summary to data/NARAKA: BLADEPOINT_query_summary.csv
Saved Query Summary to data/Destiny 2_query_summary.csv
Saved Query Summary to data/Tom C

Saved Query Summary to data/Aim Lab_query_summary.csv
Saved Query Summary to data/Eternal Return_query_summary.csv
Saved Query Summary to data/Mount & Blade: Warband_query_summary.csv
Saved Query Summary to data/Total War: ROME II - Emperor Edition_query_summary.csv
Saved Query Summary to data/Horizon Zero Dawn_query_summary.csv
Saved Query Summary to data/Dyson Sphere Program_query_summary.csv
Saved Query Summary to data/Goose Goose Duck_query_summary.csv
Saved Query Summary to data/American Truck Simulator_query_summary.csv
Saved Query Summary to data/Days Gone_query_summary.csv
Saved Query Summary to data/Hollow Knight_query_summary.csv
Saved Query Summary to data/Sekiro™: Shadows Die Twice_query_summary.csv
Saved Query Summary to data/Green Hell_query_summary.csv
Saved Query Summary to data/Fallout 76_query_summary.csv
Saved Query Summary to data/Conqueror's Blade_query_summary.csv
Saved Query Summary to data/NGU IDLE_query_summary.csv
Saved Query Summary to data/It Takes Two_query

In [73]:
for k,v in steamcharts_dict.items():
    r = requests.get(f'https://store.steampowered.com/app/{k}/{v}/')
    soup = BeautifulSoup(r.content, 'html.parser')
    tags = soup.find_all('a', class_='app_tag' )
    tag_list = [tag.text.strip('\r\n\t') for tag in tags]

730 Counter-Strike: Global Offensive
570 Dota 2
1599340 Lost Ark
578080 PUBG: BATTLEGROUNDS
1245620 ELDEN RING
1172470 Apex Legends
271590 Grand Theft Auto V
252490 Rust
440 Team Fortress 2
1569040 Football Manager 2022
431960 Wallpaper Engine
1418630 Dread Hunger
236390 War Thunder
1623660 MIR4
1203220 NARAKA: BLADEPOINT
1085660 Destiny 2
359550 Tom Clancy's Rainbow Six Siege
346110 ARK: Survival Evolved
230410 Warframe
1506830 FIFA 22
1449850 Yu-Gi-Oh!  Master Duel
289070 Sid Meier's Civilization VI
394360 Hearts of Iron IV
920210 LEGO® Star Wars™: The Skywalker Saga
251570 7 Days to Die
1794680 Vampire Survivors
221100 DayZ
381210 Dead by Daylight
304930 Unturned
227300 Euro Truck Simulator 2
444200 World of Tanks Blitz
4000 Garry's Mod
105600 Terraria
413150 Stardew Valley
252950 Rocket League
1063730 New World
582660 Black Desert
480 Spacewar
1172620 Sea of Thieves
1248130 Farming Simulator 22
218620 PAYDAY 2
236850 Europa Universalis IV
1174180 Red Dead Redemption 2
438100 VRChat

In [85]:
r = requests.get('https://store.steampowered.com/app/1272320/Diplomacy_is_Not_an_Option/')
soup = BeautifulSoup(r.content, 'html.parser')
# print(soup.prettify())
tags = soup.find_all('a', class_='app_tag' )

In [108]:
def get_tags(appid, game_title):
    """
    Returns the list of genre tags (e.g., action, sports, etc) associated with a game
    """
    appid = str(appid)
    r = requests.get(f'https://store.steampowered.com/app/{appid}/{game_title}/')
    soup = BeautifulSoup(r.content, 'html.parser')
    tags = soup.find_all('a', class_='app_tag' )
    tag_list = [tag.text.strip('\r\n\t') for tag in tags]
    return tag_list

a = get_tags('1272320', 'Diplomacy_is_Not_an_Option')

In [114]:
# Loop through each games list of tags, choose first tag that matches the major category,
# and then return a dict of game_title and tag
genres = ['action', 'adventure', 'rpg', 'sports', 'strategy', 'simulation']
for tag in a:
    if tag.lower() in genres:
        print(tag)

Strategy
Action


In [115]:
tag_dict = dict()
for k, v in steamcharts_dict.items():
    tags = get_tags(k, v)
    tag_dict[v] = tags

In [116]:
tag_dict

{'Counter-Strike: Global Offensive': ['FPS',
  'Shooter',
  'Multiplayer',
  'Competitive',
  'Action',
  'Team-Based',
  'eSports',
  'Tactical',
  'First-Person',
  'PvP',
  'Online Co-Op',
  'Co-op',
  'Strategy',
  'Military',
  'War',
  'Difficult',
  'Trading',
  'Realistic',
  'Fast-Paced',
  'Moddable'],
 'Dota 2': ['Free to Play',
  'MOBA',
  'Multiplayer',
  'Strategy',
  'eSports',
  'Team-Based',
  'Competitive',
  'Action',
  'Online Co-Op',
  'PvP',
  'Difficult',
  'Co-op',
  'RTS',
  'Tower Defense',
  'Fantasy',
  'RPG',
  'Character Customization',
  'Replay Value',
  'Action RPG',
  'Simulation'],
 'Lost Ark': ['MMORPG',
  'Action RPG',
  'Free to Play',
  'Hack and Slash',
  'Action',
  'RPG',
  'Multiplayer',
  'Massively Multiplayer',
  'Adventure',
  'PvP',
  'PvE',
  'Fantasy',
  'Character Customization',
  'Isometric',
  'Sexual Content',
  'Online Co-Op',
  'Class-Based',
  'Violent',
  'Gore',
  'Singleplayer'],
 'PUBG: BATTLEGROUNDS': ['Survival',
  'Shoote

In [118]:
tag_output = dict()
genres = ['action', 'adventure', 'rpg', 'sports', 'strategy', 'simulation']

for k, v in tag_dict.items():
    category = None
    for tag in v:
        if tag.lower() in genres:
            category = tag.lower()
            break
            
    tag_output[k] = category


In [119]:
tag_output

{'Counter-Strike: Global Offensive': 'action',
 'Dota 2': 'strategy',
 'Lost Ark': 'action',
 'PUBG: BATTLEGROUNDS': 'action',
 'ELDEN RING': 'rpg',
 'Apex Legends': 'action',
 'Grand Theft Auto V': 'action',
 'Rust': 'adventure',
 'Team Fortress 2': 'action',
 'Football Manager 2022': 'sports',
 'Wallpaper Engine': 'action',
 'Dread Hunger': 'strategy',
 'War Thunder': 'simulation',
 'MIR4': 'rpg',
 'NARAKA: BLADEPOINT': 'action',
 'Destiny 2': 'action',
 "Tom Clancy's Rainbow Six Siege": 'action',
 'ARK: Survival Evolved': 'adventure',
 'Warframe': 'rpg',
 'FIFA 22': 'sports',
 'Yu-Gi-Oh!  Master Duel': 'strategy',
 "Sid Meier's Civilization VI": 'strategy',
 'Hearts of Iron IV': 'strategy',
 'LEGO® Star Wars™: The Skywalker Saga': 'adventure',
 '7 Days to Die': 'simulation',
 'Vampire Survivors': 'action',
 'DayZ': 'action',
 'Dead by Daylight': 'action',
 'Unturned': 'adventure',
 'Euro Truck Simulator 2': 'simulation',
 'World of Tanks Blitz': 'action',
 "Garry's Mod": 'simulation

In [131]:
tags_df = pd.DataFrame.from_dict(tag_output, orient='index')
tags_df = tags_df.reset_index()

In [134]:
tags_df.rename(columns={'index':'game_title'}, inplace=True)

In [135]:
df= pd.read_csv('./data/steamcharts_combined.csv')
df

C:\Users\Patrick\AppData\Local\Temp\ipykernel_6260\3046597197.py:1: DtypeWarning: Columns (8,13,14,15,22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('./data/steamcharts_combined.csv')


,Unnamed: 0.1,Unnamed: 0,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,...,timestamp_dev_responded,developer_response,genre,num_reviews,review_score,review_score_desc,total_positive,total_negative,total_reviews,Game Title
0,0.0,0.0,113699669.0,"{'steamid': '76561199208725134', 'num_games_ow...",english,It's better than most survival games out there...,1.649697e+09,1.649697e+09,True,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,113699430.0,"{'steamid': '76561198099797173', 'num_games_ow...",english,Bonding session with Bros and Fam!,1.649696e+09,1.649696e+09,True,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2.0,113698304.0,"{'steamid': '76561199085824796', 'num_games_ow...",english,It was good for 7 days,1.649695e+09,1.649695e+09,True,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,3.0,113696677.0,"{'steamid': '76561198013393537', 'num_games_ow...",english,The best combination of all the games I love t...,1.649692e+09,1.649692e+09,True,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.0,113692215.0,"{'steamid': '76561197986629897', 'num_games_ow...",english,It's a sandbox zombie survival game with a sim...,1.649686e+09,1.649686e+09,True,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
len(df.game_title.unique())

272

In [140]:
len(tags_df.game_title.unique())

200

In [141]:
steamcharts_with_genre = pd.merge(df, tags_df)

In [142]:
steamcharts_with_genre.to_csv('steamcharts_with_genre.csv')